In [1]:
import cv2
import numpy as np
import cvlib as cv

from collections import deque
from timeit import default_timer as timer
from cvlib.object_detection import draw_bbox

# Catch frame from webcam
camera = cv2.VideoCapture(1)

# Define variables for hight and width shape of the frames
HEIGH, WIDTH = 300, 400   

# Scale to decrease the frame size
SCALE = 1

# Initializing deque object for center points of the detected object
points = deque(maxlen=50)
# Define the background
last_frame = np.zeros((HEIGH, WIDTH,3) , np.uint8)
background = np.zeros((HEIGH, WIDTH,3) , np.uint8)

# Variable counting how many time we are tracking after the objects
frames_tracking_counter = 0

# Variable counting how many time we are trying to detect objects
counter_frames_predictions = 0

# Variable counting how many object we detected in every iteration
object_detected_counter = 0

# Variable store the system status of tracking or not tracking
tracking_on = False

# Restart timer for FPS
fps_start = timer()    
    
# Increasing FPS counter
counter_fps = 0

# Define FPS Variable
FPS = 0

########################################################################################################################
    
# Function return 3-Dimension frame
def expands_dimensions(frame):
    
    new_image = np.zeros((HEIGH, WIDTH, 3), np.uint8)
    new_image[:, :, 0] = frame
    new_image[:, :, 1] = frame
    new_image[:, :, 2] = frame
    
    return new_image

# Convert frame from rgb to gray
def gray_frame(frame_rgb):
    
    # Converting captured frame to GRAY by OpenCV function    
    gray_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    
    frame_gray = np.zeros(frame_rgb.shape, np.uint8)
    frame_gray[:,:,0] = gray_frame
    frame_gray[:,:,1] = gray_frame
    frame_gray[:,:,2] = gray_frame
    
    return frame_gray

# Function define the mask
def mask(frame_rgb, background):
    
    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    
    # Create one more frame with Gaussian blur
    frame_gray = cv2.GaussianBlur(frame_gray, (25, 25), 0)  

    # Converting captured frame to GRAY by OpenCV function        
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    background = cv2.GaussianBlur(background, (25, 25), 0)    
    
    # Return mask to detect change between two frames   
    abs_diff = cv2.absdiff(frame_gray, background)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, mask = cv2.threshold(abs_diff, 20, 255, cv2.THRESH_BINARY)
    
    dilated_mask = cv2.dilate(mask, None, iterations = 5)

    # Expend mask dimension to 3 dimension
    mask_frame = expands_dimensions(dilated_mask)        
    
    return mask_frame

# Function manage the frames reader variables like efps etc'
def reader_manger(fps_start, counter_fps, tracking_on):
    
    # Variable says if keep reading frame or quit
    quit = False

    # Stopping the timer for FPS
    fps_stop = timer()

    # Print FPS every 1 second
    if 1.0 <= fps_stop - fps_start:

        # Define FPS
        FPS = counter_fps

        # Reset FPS counter
        counter_fps = 0

        # Restart timer for FPS
        fps_start = timer()       

    # Function waits for key to be pressed    
    key = cv2.waitKey(1) % 256

    # If 'n' is pressed, we catchs the frame and define it as the background
    if key == ord('n'):
        tracking_on = False

    # If 'q' key is pressed then quit from app
    if key == ord('q'):
        quit = True   

    return FPS, counter_fps, tracking_on, quit

# Function display the frames on the screen in one window
def display_windows(frame_rgb, frame_gray, frame_mask):   
        
    # Create left window    
    main_window = np.hstack((frame_rgb, frame_gray, frame_mask))

    # Plotting all the frames in one window
    cv2.imshow("Main_Window", main_window)     

# Function create 3 frames from the frame we read
def preproccess_frames(frame, last_frame):
    
    # Resize the main frame to (WIDTH, HEIGH) shape
    frame = cv2.resize(frame, (WIDTH, HEIGH))
        
    # Copy frame to work with deffrent variable
    frame_rgb = frame.copy()
    
    # Return gray frame in 3 dimension
    frame_gray = gray_frame(frame_rgb)
    
    # Return mask frame 
    frame_mask = mask(frame_rgb, last_frame)
    
    # Define last frame
    last_frame = frame_rgb.copy()
    
    return frame, frame_rgb, frame_gray, frame_mask, last_frame

# Function manage the detection and return status and coordinates
def detection_manager(frame, frame_rgb):

    # Store the bounding boxes with the new coordinates in a list
    boxes = []
    
    # Variable count how many objects we detect
    num_of_objects = 0

    # Function return all scores of model predictions
    bounding_boxes, detected_labels, scores = cv.detect_common_objects(frame_rgb)

    # Check if we succeeded to detect objects
    if 0 < len(bounding_boxes):
        
        # Scaling the bounding boxes back to original main frame size
        for box in bounding_boxes:

            # Increase the objects counter
            num_of_objects += 1

            # Create new list of bounding boxes that fit to main frame size
            (x_min, y_min, x_max, y_max) = [int(a) for a in box]
            
            # bounding_boxes contain x1, y1, x2, y2, coordinates and not width and heigh
            bounding_box = np.array([x_min*SCALE, y_min*SCALE, x_max*SCALE, y_max*SCALE])
            boxes.append(bounding_box)

            # Value means we start tracking after the objects
            tracking_status = "Start tracking"
    else:      
        # Value means no tracking need and have to try detect again
        tracking_status = "End tracking"
            
    return boxes, detected_labels, scores, num_of_objects, tracking_status

In [2]:
# Loop reading frame by frame and processing them
while True:
    
    # Increasing FPS counter
    counter_fps += 1
    
    # Capturing frames one-by-one from camera
    ret, frame = camera.read()

    # If the frame was not retrieved then we break the loop
    if not ret or frame is None:
        break

    # Function return 3 diffrent kind of frames
    frame, frame_rgb, frame_gray, frame_mask, last_frame = preproccess_frames(frame, last_frame)
    
    if tracking_on:
        pass
    else:

        # Increase prediction counter
        counter_frames_predictions += 1
        
        # Function manage the detection part and return coordinates of drawing   
        bounding_boxes, detected_labels, scores, object_detected_counter, tracking_status = detection_manager(frame, frame_rgb)
        
        # Function draw boxes around the detected objects
        frame = draw_bbox(frame, bounding_boxes, detected_labels, scores)
        
        
    
    
    # Display all frames in one window
    display_windows(frame, frame_gray, frame_mask)

    # Function manage the frames reader variables
    FPS, counter_fps, tracking_on, quit = reader_manger(fps_start, counter_fps, tracking_on)

    # If quit is true so we stop read frames
    if quit == True:
        break

# Releasing camera
camera.release()

# Destroying all opened OpenCV windows
cv2.destroyAllWindows()      